# Validation

> Enforcing daisyUI 5 usage rules and best practices

In [ ]:
#| default_exp core.validation

In [ ]:
#| export
from typing import List, Dict, Optional, Union, Any, Tuple, Set
from dataclasses import dataclass, field
from enum import Enum
import re
from cjm_fasthtml_daisyui.core.base import DaisyComponent
from cjm_fasthtml_daisyui.core.colors import SemanticColor
from cjm_tailwind_utils.all import TailwindBuilder

## Validation Framework

This module provides validation and enforcement of daisyUI 5 usage rules to ensure components follow best practices and maintain consistency.

In [ ]:
#| export
class ValidationLevel(str, Enum):
    """Severity levels for validation messages"""
    ERROR = "error"      # Must be fixed
    WARNING = "warning"  # Should be fixed
    INFO = "info"       # Suggestions for improvement
    
    def symbol(
        self
    ) -> str:  # TODO: Add return description
        """Get symbol for this level"""
        symbols = {
            self.ERROR: "❌",
            self.WARNING: "⚠️",
            self.INFO: "ℹ️"
        }
        return symbols[self]

In [ ]:
#| export
@dataclass
class ValidationMessage:
    """A single validation message"""
    level: ValidationLevel
    message: str
    rule: str  # Which daisyUI rule this relates to
    suggestion: Optional[str] = None
    context: Optional[Dict[str, Any]] = None
    
    def __str__(
        self
    ) -> str:  # TODO: Add return description
        """Format as string"""
        parts = [f"{self.level.symbol()} [{self.level.value}] {self.message}"]
        if self.suggestion:
            parts.append(f"   Suggestion: {self.suggestion}")
        if self.rule:
            parts.append(f"   Rule: {self.rule}")
        return "\n".join(parts)

In [ ]:
#| export
@dataclass
class ValidationResult:
    """Result of validating a component or set of classes"""
    messages: List[ValidationMessage] = field(default_factory=list)
    
    @property
    def is_valid(
        self
    ) -> bool:  # TODO: Add return description
        """Check if validation passed (no errors)"""
        return not any(msg.level == ValidationLevel.ERROR for msg in self.messages)
    
    @property
    def has_warnings(
        self
    ) -> bool:  # TODO: Add return description
        """Check if there are any warnings"""
        return any(msg.level == ValidationLevel.WARNING for msg in self.messages)
    
    def add(self, level: ValidationLevel, message: str, rule: str, 
            suggestion: Optional[str] = None, context: Optional[Dict[str, Any]] = None):
        """Add a validation message"""
        self.messages.append(ValidationMessage(
            level=level,
            message=message,
            rule=rule,
            suggestion=suggestion,
            context=context
        ))
    
    def get_by_level(
        self,
        level: ValidationLevel  # TODO: Add description
    ) -> List[ValidationMessage]:  # TODO: Add return description
        """Get all messages of a specific level"""
        return [msg for msg in self.messages if msg.level == level]
    
    def summary(
        self
    ) -> str:  # TODO: Add return description
        """Get a summary of validation results"""
        errors = len(self.get_by_level(ValidationLevel.ERROR))
        warnings = len(self.get_by_level(ValidationLevel.WARNING))
        info = len(self.get_by_level(ValidationLevel.INFO))
        
        parts = []
        if errors:
            parts.append(f"{errors} error(s)")
        if warnings:
            parts.append(f"{warnings} warning(s)")
        if info:
            parts.append(f"{info} info message(s)")
        
        if not parts:
            return "✅ Validation passed"
        
        return f"Validation completed with: {', '.join(parts)}"
    
    def __str__(
        self
    ) -> str:  # TODO: Add return description
        """Format all messages"""
        if not self.messages:
            return "✅ No validation issues found"
        
        lines = [self.summary(), ""]
        for msg in self.messages:
            lines.append(str(msg))
            lines.append("")
        
        return "\n".join(lines)

## Core Validation Rules

Implementation of daisyUI 5 usage rules as validators:

In [ ]:
#| export
class DaisyUIRules:
    """
    Enforces daisyUI 5 usage rules
    
    Based on the official daisyUI 5 usage rules:
    1. Components need component class + optional part/modifier classes
    2. Customization via Tailwind utilities (use ! for specificity issues)
    3. Create custom components with Tailwind if not in daisyUI
    4. Responsive layouts should use Tailwind responsive prefixes
    5. Use daisyUI or Tailwind classes only
    6. Prefer semantic colors over hardcoded Tailwind colors
    7. Avoid hardcoded colors that won't adapt to themes
    """
    
    # Patterns for detecting various class types
    TAILWIND_COLOR_PATTERN = re.compile(
        r'\b(text|bg|border|ring|fill|stroke)-(slate|gray|zinc|neutral|stone|'
        r'red|orange|amber|yellow|lime|green|emerald|teal|cyan|sky|blue|indigo|'
        r'violet|purple|fuchsia|pink|rose)-\d{2,3}\b'
    )
    
    DAISY_COMPONENT_PATTERN = re.compile(
        r'\b(btn|badge|card|modal|drawer|dropdown|menu|navbar|footer|hero|'
        r'alert|avatar|breadcrumbs|chat|checkbox|collapse|countdown|diff|'
        r'divider|dock|fieldset|file-input|filter|indicator|input|join|kbd|'
        r'label|link|list|loading|mask|mockup|pagination|progress|radio|'
        r'range|rating|select|skeleton|stack|stat|steps|swap|tab|table|'
        r'textarea|timeline|toast|toggle|validator)\b'
    )
    
    RESPONSIVE_PATTERN = re.compile(r'\b(sm|md|lg|xl|2xl):')
    
    ARBITRARY_VALUE_PATTERN = re.compile(r'\[.+\]')
    
    @classmethod
    def validate_color_usage(
        cls,  # TODO: Add type hint and description
        classes: Union[str, List[str]]  # TODO: Add description
    ) -> ValidationResult:  # TODO: Add return description
        """
        Validate color usage (Rules 6-9)
        
        Checks for:
        - Hardcoded Tailwind colors that won't adapt to themes
        - Suggests semantic colors instead
        """
        result = ValidationResult()
        
        if isinstance(classes, str):
            classes = classes.split()
        
        for class_name in classes:
            match = cls.TAILWIND_COLOR_PATTERN.match(class_name)
            if match:
                utility_type = match.group(1)
                color_name = match.group(2)
                
                # Neutral colors are sometimes acceptable
                if color_name in ['neutral', 'slate', 'gray', 'zinc', 'stone']:
                    result.add(
                        ValidationLevel.INFO,
                        f"Using Tailwind color '{class_name}' which may not adapt to themes",
                        "Rule 6-8: Prefer semantic colors",
                        f"Consider using semantic colors like '{utility_type}-base-content' or '{utility_type}-neutral'",
                        {"class": class_name, "utility": utility_type, "color": color_name}
                    )
                else:
                    result.add(
                        ValidationLevel.WARNING,
                        f"Using hardcoded Tailwind color '{class_name}' that won't adapt to themes",
                        "Rule 7-8: Avoid non-semantic colors",
                        f"Use semantic colors like '{utility_type}-primary' or '{utility_type}-error' instead",
                        {"class": class_name, "utility": utility_type, "color": color_name}
                    )
        
        return result
    
    @classmethod
    def validate_component_structure(
        cls,  # TODO: Add type hint and description
        component: DaisyComponent  # TODO: Add description
    ) -> ValidationResult:  # TODO: Add return description
        """
        Validate component structure (Rule 1)
        
        Checks for:
        - Proper component class naming
        - Valid modifier usage
        """
        result = ValidationResult()
        
        try:
            component_class = component.component_class()
            classes = component.build_classes().split()
            
            # Check if component class is present
            if component_class not in classes:
                result.add(
                    ValidationLevel.ERROR,
                    f"Component class '{component_class}' not found in generated classes",
                    "Rule 1: Components need base class",
                    "Ensure component_class() is included in build_classes()"
                )
            
            # Check for valid daisyUI component
            has_daisy_component = any(
                cls.DAISY_COMPONENT_PATTERN.match(c) for c in classes
            )
            
            if not has_daisy_component and component_class not in ['example']:  # Allow test components
                result.add(
                    ValidationLevel.INFO,
                    f"Component '{component_class}' may not be a standard daisyUI component",
                    "Rule 3: Use Tailwind for custom components",
                    "If this is a custom component, ensure it follows daisyUI patterns"
                )
        
        except NotImplementedError:
            result.add(
                ValidationLevel.ERROR,
                "Component does not implement component_class() method",
                "Rule 1: Components need base class",
                "All DaisyComponent subclasses must implement component_class()"
            )
        
        return result
    
    @classmethod
    def suggest_responsive(
        cls,  # TODO: Add type hint and description
        classes: Union[str, List[str]]  # TODO: Add description
    ) -> ValidationResult:  # TODO: Add return description
        """
        Suggest responsive improvements (Rule 4)
        
        Checks for:
        - Components that might benefit from responsive modifiers
        """
        result = ValidationResult()
        
        if isinstance(classes, str):
            classes = classes.split()
        
        # Components that commonly need responsive behavior
        responsive_components = {
            'navbar': 'Consider sm:navbar-horizontal for mobile-first design',
            'footer': 'Consider sm:footer-horizontal for responsive layout',
            'menu': 'Consider lg:menu-horizontal for responsive navigation',
            'join': 'Consider lg:join-horizontal for responsive grouping',
            'stats': 'Consider sm:stats-horizontal for mobile layout',
            'steps': 'Consider sm:steps-vertical for mobile layout',
            'tabs': 'Consider responsive tab sizing with sm:tab-lg',
            'grid': 'Consider responsive grid columns like sm:grid-cols-2 lg:grid-cols-3',
            'flex': 'Consider responsive flex direction like sm:flex-row'
        }
        
        for class_name in classes:
            for component, suggestion in responsive_components.items():
                if component in class_name and not any(cls.RESPONSIVE_PATTERN.match(c) for c in classes):
                    result.add(
                        ValidationLevel.INFO,
                        f"Component '{component}' might benefit from responsive modifiers",
                        "Rule 4: Use responsive prefixes",
                        suggestion,
                        {"component": component}
                    )
                    break
        
        return result
    
    @classmethod
    def validate_custom_css(cls, classes: Union[str, List[str]], 
                           allow_arbitrary: bool = True) -> ValidationResult:
        """
        Validate against custom CSS (Rules 5-7)
        
        Checks for:
        - Non-daisyUI/Tailwind classes
        - Arbitrary values (which are allowed but tracked)
        """
        result = ValidationResult()
        
        if isinstance(classes, str):
            classes = classes.split()
        
        # Known valid prefixes for Tailwind utilities
        valid_prefixes = {
            # Layout
            'container', 'block', 'inline', 'flex', 'grid', 'table', 'hidden',
            # Spacing
            'p', 'px', 'py', 'pt', 'pr', 'pb', 'pl', 'm', 'mx', 'my', 'mt', 'mr', 'mb', 'ml',
            'space', 'gap',
            # Sizing
            'w', 'h', 'min', 'max', 'size',
            # Typography
            'text', 'font', 'leading', 'tracking', 'decoration',
            # Colors (handled separately)
            'bg', 'text', 'border', 'ring', 'fill', 'stroke',
            # Effects
            'shadow', 'opacity', 'blur', 'brightness',
            # Transitions
            'transition', 'duration', 'ease', 'delay',
            # Transforms
            'scale', 'rotate', 'translate', 'skew',
            # Interactivity
            'cursor', 'select', 'resize', 'scroll',
            # Flexbox & Grid
            'justify', 'items', 'content', 'self', 'place', 'order',
            # Borders
            'border', 'rounded', 'ring', 'divide',
            # Positioning
            'static', 'fixed', 'absolute', 'relative', 'sticky', 'inset', 'top', 'right', 'bottom', 'left', 'z',
            # Display
            'overflow', 'object', 'overscroll',
            # Filters
            'filter', 'backdrop',
            # Tables
            'table', 'caption',
            # Animations
            'animate',
            # Pseudo-classes
            'hover', 'focus', 'active', 'visited', 'disabled', 'first', 'last', 'odd', 'even',
            # Breakpoints
            'sm', 'md', 'lg', 'xl', '2xl',
            # Dark mode
            'dark',
            # Important modifier
            '!',
        }
        
        # Check each class
        for class_name in classes:
            # Skip empty classes
            if not class_name:
                continue
            
            # Check for arbitrary values
            if cls.ARBITRARY_VALUE_PATTERN.search(class_name):
                if allow_arbitrary:
                    continue  # Arbitrary values are allowed
                else:
                    result.add(
                        ValidationLevel.INFO,
                        f"Using arbitrary value in '{class_name}'",
                        "Rule 7: Prefer defined utilities",
                        "Consider using predefined Tailwind utilities when possible"
                    )
                continue
            
            # Check if it's a known daisyUI component/modifier
            if cls.DAISY_COMPONENT_PATTERN.search(class_name):
                continue
            
            # Check if it starts with a valid Tailwind prefix
            is_valid_tailwind = False
            for prefix in valid_prefixes:
                if class_name.startswith(prefix + '-') or class_name.startswith(prefix + ':') or class_name == prefix:
                    is_valid_tailwind = True
                    break
            
            # Check for responsive prefix
            if ':' in class_name:
                prefix_part = class_name.split(':')[0]
                if prefix_part in valid_prefixes:
                    is_valid_tailwind = True
            
            # Check for important modifier
            if class_name.startswith('!'):
                check_class = class_name[1:]
                for prefix in valid_prefixes:
                    if check_class.startswith(prefix + '-') or check_class == prefix:
                        is_valid_tailwind = True
                        break
            
            if not is_valid_tailwind and not class_name.startswith('daisy-'):  # Allow prefixed classes
                result.add(
                    ValidationLevel.WARNING,
                    f"Class '{class_name}' may not be a valid daisyUI or Tailwind class",
                    "Rule 5-6: Use only daisyUI/Tailwind classes",
                    "Ensure this is a valid utility class or consider creating a custom component"
                )
        
        return result
    
    @classmethod
    def validate_all(cls, component: DaisyComponent, 
                    check_responsive: bool = True) -> ValidationResult:
        """
        Run all validation rules on a component
        
        Args:
            component: The component to validate
            check_responsive: Whether to check for responsive suggestions
            
        Returns:
            Combined validation results
        """
        result = ValidationResult()
        
        # Get component classes
        try:
            classes = component.build_classes()
        except Exception as e:
            result.add(
                ValidationLevel.ERROR,
                f"Failed to build component classes: {str(e)}",
                "General validation error",
                "Fix component implementation"
            )
            return result
        
        # Run individual validators
        validations = [
            cls.validate_component_structure(component),
            cls.validate_color_usage(classes),
            cls.validate_custom_css(classes),
        ]
        
        if check_responsive:
            validations.append(cls.suggest_responsive(classes))
        
        # Combine results
        for validation in validations:
            result.messages.extend(validation.messages)
        
        return result

## Component-Specific Validators

Additional validators for specific component types:

In [ ]:
#| export
class ComponentValidators:
    """Validators for specific component types"""
    
    @staticmethod
    def validate_button(
        classes: str  # TODO: Add description
    ) -> ValidationResult:  # TODO: Add return description
        """Validate button-specific rules"""
        result = ValidationResult()
        class_list = classes.split()
        
        if 'btn' in class_list:
            # Check for conflicting styles
            styles = ['btn-outline', 'btn-dash', 'btn-soft', 'btn-ghost', 'btn-link']
            style_count = sum(1 for style in styles if style in class_list)
            
            if style_count > 1:
                result.add(
                    ValidationLevel.WARNING,
                    "Button has multiple conflicting styles",
                    "Component best practices",
                    "Use only one style modifier per button",
                    {"styles": [s for s in styles if s in class_list]}
                )
            
            # Check for size consistency
            sizes = ['btn-xs', 'btn-sm', 'btn-md', 'btn-lg', 'btn-xl']
            size_count = sum(1 for size in sizes if size in class_list)
            
            if size_count > 1:
                result.add(
                    ValidationLevel.ERROR,
                    "Button has multiple size modifiers",
                    "Component structure",
                    "Use only one size modifier per button",
                    {"sizes": [s for s in sizes if s in class_list]}
                )
        
        return result
    
    @staticmethod
    def validate_form_control(
        classes: str,  # TODO: Add description
        component_type: str  # TODO: Add description
    ) -> ValidationResult:  # TODO: Add return description
        """Validate form control components (input, select, textarea)"""
        result = ValidationResult()
        class_list = classes.split()
        
        # Check for label association
        if component_type in class_list and 'label' not in classes:
            result.add(
                ValidationLevel.INFO,
                f"{component_type.title()} without associated label",
                "Accessibility best practice",
                f"Consider wrapping {component_type} with a label element or using label with 'for' attribute"
            )
        
        # Check for validation classes without validator
        if any(c in class_list for c in ['input-error', 'input-success']) and 'validator' not in class_list:
            result.add(
                ValidationLevel.INFO,
                "Using validation state without validator class",
                "Component usage",
                "Consider adding 'validator' class for automatic validation styling"
            )
        
        return result
    
    @staticmethod
    def validate_modal(
        classes: str,  # TODO: Add description
        has_trigger: bool = False  # TODO: Add description
    ) -> ValidationResult:  # TODO: Add return description
        """Validate modal component structure"""
        result = ValidationResult()
        class_list = classes.split()
        
        if 'modal' in class_list:
            # Check for required parts
            if 'modal-box' not in classes:
                result.add(
                    ValidationLevel.ERROR,
                    "Modal missing required 'modal-box' part",
                    "Component structure",
                    "Add a div with class 'modal-box' inside the modal"
                )
            
            # Check for trigger mechanism
            if not has_trigger and 'modal-open' not in class_list:
                result.add(
                    ValidationLevel.INFO,
                    "Modal without visible trigger mechanism",
                    "Component usage",
                    "Ensure modal has a trigger button or uses modal-toggle/dialog element"
                )
        
        return result

## Validation Helpers

Utility functions for common validation tasks:

In [ ]:
#| export
def validate_classes(classes: Union[str, List[str]], 
                    strict: bool = False) -> ValidationResult:
    "Quick validation of CSS classes"
    result = DaisyUIRules.validate_color_usage(classes)
    css_result = DaisyUIRules.validate_custom_css(classes)
    
    # Combine results
    result.messages.extend(css_result.messages)
    
    # Upgrade warnings to errors if strict mode
    if strict:
        for msg in result.messages:
            if msg.level == ValidationLevel.WARNING:
                msg.level = ValidationLevel.ERROR
    
    return result

In [ ]:
#| export
def assert_valid(component: DaisyComponent, 
                message: str = "Component validation failed"):
    "Assert that a component is valid, raising an exception if not"
    result = DaisyUIRules.validate_all(component, check_responsive=False)
    
    if not result.is_valid:
        errors = result.get_by_level(ValidationLevel.ERROR)
        error_messages = [f"  - {error.message}" for error in errors]
        full_message = f"{message}:\n" + "\n".join(error_messages)
        raise AssertionError(full_message)

In [ ]:
#| export
class ValidatedComponent(DaisyComponent):
    """
    Base class for components with automatic validation
    
    Subclasses will have their output validated automatically
    to ensure compliance with daisyUI rules.
    """
    
    def __init__(self, *args, validate_on_init: bool = True, **kwargs):
        "TODO: Add function description"
        super().__init__(*args, **kwargs)
        self.validate_on_init = validate_on_init
        
        if validate_on_init:
            self._validate()
    
    def _validate(
        self
    ): # TODO: Add type hint
        """Run validation and handle results"""
        result = DaisyUIRules.validate_all(self, check_responsive=False)
        
        # Log warnings in development
        if result.has_warnings:
            for warning in result.get_by_level(ValidationLevel.WARNING):
                print(f"Validation Warning: {warning.message}")
        
        # Raise errors
        if not result.is_valid:
            assert_valid(self)
    
    def build_classes(
        self
    ) -> str:  # TODO: Add return description
        """Build classes with validation"""
        classes = super().build_classes()
        
        # Validate the generated classes
        if self.validate_on_init:
            class_result = validate_classes(classes)
            if not class_result.is_valid:
                errors = class_result.get_by_level(ValidationLevel.ERROR)
                raise ValueError(f"Invalid classes generated: {errors[0].message}")
        
        return classes

## Usage Examples

Examples of using the validation framework:

### Validating CSS Classes

In [ ]:
# Example: Validate color usage - hardcoded colors that won't adapt to themes
# Using TailwindBuilder to create classes with non-semantic colors
classes_with_issues = (TailwindBuilder()
                      .add_class("btn btn-primary")
                      .bg("red-500")     # Non-semantic color
                      .text(color="white")  # Non-semantic color
                      .build())

result = validate_classes(classes_with_issues)
print(f"Classes being validated: {classes_with_issues}")
print(result)

Classes being validated: btn btn-primary bg-red-500 text-white
Validation completed with: 1 warning(s)

⚠️ [warning] Using hardcoded Tailwind color 'bg-red-500' that won't adapt to themes
   Suggestion: Use semantic colors like 'bg-primary' or 'bg-error' instead
   Rule: Rule 7-8: Avoid non-semantic colors



In [ ]:
# Example: Validate semantic colors are preferred
# TailwindBuilder's text() method accepts daisyUI semantic colors,
# but bg() only accepts standard Tailwind colors, so we use add_class for bg
good_classes = (TailwindBuilder()
                .add_class("btn btn-primary")
                .add_class("bg-base-200")         # daisyUI semantic bg color
                .text(color="base-content")       # daisyUI semantic text color works!
                .build())

result = validate_classes(good_classes)
print(f"Classes being validated: {good_classes}")
print(result)

Classes being validated: btn btn-primary bg-base-200 text-base-content
✅ No validation issues found


### Validating Components

In [ ]:
# Example: Create a test component
class TestButton(DaisyComponent):
    def component_class(self) -> str:
        return "btn"
    
    def supports_color(self) -> bool:
        return True
    
    def supports_size(self) -> bool:
        return True

# Create button with potential issues using TailwindBuilder
# Note: bg() only accepts standard Tailwind colors, so we add the non-semantic color differently
button = TestButton(
    color=SemanticColor.PRIMARY,
    size="lg",
    cls=(TailwindBuilder()
         .bg("red-500")        # Non-semantic Tailwind color
         .text(color="white")  # Non-semantic color
         .build())
)

# Validate the component
validation_result = DaisyUIRules.validate_all(button)
print(validation_result)

Validation completed with: 1 warning(s)

⚠️ [warning] Using hardcoded Tailwind color 'bg-red-500' that won't adapt to themes
   Suggestion: Use semantic colors like 'bg-primary' or 'bg-error' instead
   Rule: Rule 7-8: Avoid non-semantic colors



### Using ValidatedComponent

In [ ]:
# Example: Component with automatic validation
class ValidatedButton(ValidatedComponent):
    def component_class(self) -> str:
        return "btn"
    
    def supports_color(self) -> bool:
        return True

# This will validate automatically on creation
try:
    # Good button - no issues
    good_button = ValidatedButton(color=SemanticColor.SUCCESS)
    print(f"Good button classes: {good_button.build_classes()}")
    
    # Button with warnings (will print warnings but not fail)
    # Using TailwindBuilder to add non-semantic color
    warning_button = ValidatedButton(
        color=SemanticColor.WARNING,
        cls=(TailwindBuilder()
             .text(color="yellow-500")  # Non-semantic color - will trigger warning
             .build())
    )
    print(f"\nWarning button classes: {warning_button.build_classes()}")
    
except AssertionError as e:
    print(f"Validation failed: {e}")

Good button classes: btn btn-success
Validation Warning: Using hardcoded Tailwind color 'text-yellow-500' that won't adapt to themes

Warning button classes: btn btn-warning text-yellow-500


### Component-Specific Validation

In [ ]:
# Example: Validate button-specific rules
# Using TailwindBuilder to create invalid button classes (multiple styles and sizes)
button_classes = (TailwindBuilder()
                  .add_class("btn btn-primary btn-outline btn-ghost")  # Multiple styles
                  .add_class("btn-lg btn-xl")                          # Multiple sizes
                  .build())

result = ComponentValidators.validate_button(button_classes)
print("Button validation:")
print(result)

# Example: Validate form control
# Creating input classes with TailwindBuilder
input_classes = (TailwindBuilder()
                 .add_class("input input-bordered input-error")
                 .build())

result = ComponentValidators.validate_form_control(input_classes, "input")
print("\nInput validation:")
print(result)

# Example: Creating properly structured button with TailwindBuilder
proper_button = (TailwindBuilder()
                 .add_class("btn btn-primary btn-lg")  # Only one style, one size
                 .shadow("lg")
                 .rounded("lg")
                 .build())

print(f"\nProperly structured button classes: {proper_button}")
result = ComponentValidators.validate_button(proper_button)
print(result)

Button validation:
Validation completed with: 1 error(s), 1 warning(s)

⚠️ [warning] Button has multiple conflicting styles
   Suggestion: Use only one style modifier per button
   Rule: Component best practices

❌ [error] Button has multiple size modifiers
   Suggestion: Use only one size modifier per button
   Rule: Component structure


Input validation:
Validation completed with: 2 info message(s)

ℹ️ [info] Input without associated label
   Suggestion: Consider wrapping input with a label element or using label with 'for' attribute
   Rule: Accessibility best practice

ℹ️ [info] Using validation state without validator class
   Suggestion: Consider adding 'validator' class for automatic validation styling
   Rule: Component usage


Properly structured button classes: btn btn-primary btn-lg shadow-lg rounded-lg
✅ No validation issues found


## Best Practices

1. **Use Semantic Colors**: Always prefer semantic colors that adapt to themes
2. **Validate Early**: Use `ValidatedComponent` base class for automatic validation
3. **Check Responsive**: Enable responsive suggestions for production components
4. **Handle Warnings**: Address warnings even if they don't block functionality
5. **Component Rules**: Follow component-specific validation rules
6. **Test Themes**: Validate components work across different themes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()